# Penalized Regression Techniques

In [1]:
# L1- Lasso Regression
# L2- Ridge Regression
# Enet- Elastic Net 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [3]:
from sklearn import datasets, linear_model
from sklearn import model_selection as cv
from scipy import linalg

In [4]:
bhdata = datasets.load_boston()

featuredata = bhdata.data

colnames = bhdata.feature_names #variable names

data = pd.DataFrame(featuredata,columns=colnames) # predictors of MEDV
data['target'] = bhdata.target #median value of housing prices

In [5]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
data.shape

(506, 14)

In [7]:
13*50

650

In [8]:
data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [9]:
#split the dataset into training and testig 
data_train, data_test = cv.train_test_split(data,test_size=0.30,random_state=34)

In [10]:
data_train.shape,data_test.shape

((354, 14), (152, 14))

In [11]:
corr = data.corr()
corr

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
CRIM,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
ZN,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
INDUS,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
CHAS,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
NOX,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
RM,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
AGE,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
DIS,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
RAD,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
TAX,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


In [14]:
XTrain = data_train.iloc[:,0:(data.shape[1]-1)]
YTrain = data_train['target']

In [15]:
XTest = data_test.iloc[:,0:(data.shape[1]-1)]
YTest = data_test['target']

In [16]:
XTrain.shape, XTest.shape

((354, 13), (152, 13))

In [17]:
#multiple linear regression model
reg = linear_model.LinearRegression(normalize=True)
reg

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [18]:
reg.fit(XTrain,YTrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [19]:
print('Coefficients: \n', np.round(reg.coef_,1))

Coefficients: 
 [ -0.1   0.1   0.1   2.5 -19.5   3.1   0.   -1.3   0.3  -0.   -1.    0.
  -0.6]


In [20]:
reg.score(XTrain,YTrain)

0.7304342787291502

In [21]:
reg.score(XTest,YTest)

0.7453812732148353

In [22]:
from scipy import stats

In [23]:
#adjusted R square
def AdjustedRSquare(model,X,Y):
    YHat = model.predict(X)
    n,k = X.shape
    sse = np.sum(np.square(YHat-Y),axis=0)
    sst = np.sum(np.square(Y-np.mean(Y)),axis=0)
    R2 = 1- sse/sst
    adjR2 = R2-(1-R2)*(float(k)/(n-k-1))
    return adjR2, R2

In [24]:
def ReturnPValue(model,X,Y):
    YHat = model.predict(X)
    n,k = X.shape
    sse = np.sum(np.square(YHat-Y),axis=0)
    x = np.hstack((np.ones((n,1)),np.matrix(X)))
    df = float(n-k-1)
    sampleVar = sse/df
    sampleVarianceX = x.T*x
    covarianceMatrix = linalg.sqrtm(sampleVar*sampleVarianceX.I)
    se = covarianceMatrix.diagonal()[1:]
    betasTstat = np.zeros(len(se))
    for i in range(len(se)):
        betasTstat[i] = model.coef_[i]/se[i]
    betasPvalue = 1- stats.t.cdf(abs(betasTstat),df)
    return betasPvalue

In [25]:
np.round(ReturnPValue(reg,XTrain,YTrain),2)

array([0.02, 0.  , 0.14, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  ])

In [26]:
reg.adjR2, reg.R2 = AdjustedRSquare(reg,XTrain,YTrain)
print (reg.adjR2, reg.R2)

0.7201273540923236 0.7304342787291502


In [27]:
def ErrorMetric(model,X,Y):
    Yhat = model.predict(X)
    MAPE = np.mean(abs(Y-Yhat)/Y)*100
    MSSE = np.mean(np.square(Y-Yhat))
    return MAPE, MSSE

In [28]:
ErrorMetric(reg,XTrain,YTrain)

(16.45621290395976, 22.789606639490117)

In [29]:
ErrorMetric(reg,XTest,YTest)

(17.698843414792957, 21.327407207278622)

In [30]:
alpha = 0.05
erroR = 1

In [31]:
reg.BetasPValue = ReturnPValue(reg,XTrain,YTrain)

In [32]:
MAPE, prevMSSE = ErrorMetric(reg,XTest,YTest)
MAPE, prevMSSE

(17.698843414792957, 21.327407207278622)

In [35]:
while (erroR > 0.001):
    ind = np.where(reg.BetasPValue>alpha)[0]
    if(len(ind)==0):
        ind = np.argmax(reg.BetasPValue)
        
    XTrain = np.delete(XTrain,ind,axis=1)
    XTest = np.delete(XTest,ind,axis=1)
    
    reg.fit(XTrain,YTrain)
    
    reg.BetasPValue = ReturnPValue(reg,XTrain, YTrain)
    MAPE, MSSE = ErrorMetric(reg,XTest,YTest)

    erroR = prevMSSE - MSSE
print (erroR, MSSE)

ValueError: ignored

In [34]:
np.round(reg.coef_,1)

array([ -0.1,   0.1,   0.1,   2.5, -19.5,   3.1,   0. ,  -1.3,   0.3,
        -0. ,  -1. ,   0. ,  -0.6])

In [ ]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
data.RAD.value_counts()

24.0    132
5.0     115
4.0     110
3.0      38
6.0      26
8.0      24
2.0      24
1.0      20
7.0      17
Name: RAD, dtype: int64

In [ ]:
RAD_Dummies = pd.get_dummies(data.RAD,prefix='RAD',drop_first=True)

In [ ]:
RAD_Dummies.head(10)

,RAD_2.0,RAD_3.0,RAD_4.0,RAD_5.0,RAD_6.0,RAD_7.0,RAD_8.0,RAD_24.0
0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0
7,0,0,0,1,0,0,0,0
8,0,0,0,1,0,0,0,0
9,0,0,0,1,0,0,0,0


In [ ]:
modeldata = pd.concat([data,RAD_Dummies],axis=1)

In [ ]:
modeldata.head(2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,...,LSTAT,target,RAD_2.0,RAD_3.0,RAD_4.0,RAD_5.0,RAD_6.0,RAD_7.0,RAD_8.0,RAD_24.0
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,...,4.98,24.0,0,0,0,0,0,0,0,0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,...,9.14,21.6,1,0,0,0,0,0,0,0


In [ ]:
df_train, df_test = cv.train_test_split(modeldata,test_size=0.30,random_state=1234)

In [ ]:
df_train.shape, df_test.shape

((354, 22), (152, 22))

In [ ]:
XTrain = df_train.iloc[:,list(range(7)) + list(range(9,13)) + list(range(15,df_train.shape[1]))].as_matrix()
YTrain = df_train['target']

/Users/pradmishra/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [ ]:
XTest = df_test.iloc[:,list(range(7)) + list(range(9,13)) + list(range(15,df_test.shape[1]))].as_matrix()
YTest = df_test['target']

/Users/pradmishra/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [ ]:
XTrain.shape,YTrain.shape

((354, 18), (354,))

In [ ]:
reg

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
regr.fit(XTrain,YTrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
regr.score(XTrain,YTrain)

0.695565343494841

In [ ]:
regr.MAPE, regr.MSSE = ErrorMetric(regr,XTest,YTest)
regr.MAPE, regr.MSSE

(18.160706724458546, 23.771389527802278)

In [ ]:
regr.adjR2, regr.R2 = AdjustedRSquare(regr,XTrain,YTrain)
print (regr.adjR2, regr.R2)

0.6792076604587429 0.695565343494841


In [ ]:
# Polynomial features
# We can try Polynomial Rergession 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [ ]:
# two kinds of polynomial features:
# base features =13
# transforming the individual features (degree 2= 13)
# creating interaction features (13C2 = 78)

In [ ]:
78+26

104

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(bhdata['data'])

In [ ]:
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501


In [ ]:
poly = PolynomialFeatures(3)

In [ ]:
poly

PolynomialFeatures(degree=3, include_bias=True, interaction_only=False,
                   order='C')

In [ ]:
Xpoly = poly.fit_transform(X)

In [ ]:
Xpoly.shape

(506, 560)

In [ ]:
print (len(Xpoly))

506


In [ ]:
x = Xpoly
Y = bhdata['target']

In [ ]:
# interaction features
featurenames = ['intercept'] + list(bhdata['feature_names'])

polyfeaturenames = [];

for i,x in enumerate(featurenames):
    for y in featurenames[i:]:
        if (x == 'intercept'):
            polyfeaturenames.append(y)
        elif (x==y):
            polyfeaturenames.append((y+'_Square'))
        else:
            polyfeaturenames.append((x+'_'+y))
print(featurenames)


['intercept', 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']


In [ ]:
print (polyfeaturenames)

['intercept', 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'CRIM_Square', 'CRIM_ZN', 'CRIM_INDUS', 'CRIM_CHAS', 'CRIM_NOX', 'CRIM_RM', 'CRIM_AGE', 'CRIM_DIS', 'CRIM_RAD', 'CRIM_TAX', 'CRIM_PTRATIO', 'CRIM_B', 'CRIM_LSTAT', 'ZN_Square', 'ZN_INDUS', 'ZN_CHAS', 'ZN_NOX', 'ZN_RM', 'ZN_AGE', 'ZN_DIS', 'ZN_RAD', 'ZN_TAX', 'ZN_PTRATIO', 'ZN_B', 'ZN_LSTAT', 'INDUS_Square', 'INDUS_CHAS', 'INDUS_NOX', 'INDUS_RM', 'INDUS_AGE', 'INDUS_DIS', 'INDUS_RAD', 'INDUS_TAX', 'INDUS_PTRATIO', 'INDUS_B', 'INDUS_LSTAT', 'CHAS_Square', 'CHAS_NOX', 'CHAS_RM', 'CHAS_AGE', 'CHAS_DIS', 'CHAS_RAD', 'CHAS_TAX', 'CHAS_PTRATIO', 'CHAS_B', 'CHAS_LSTAT', 'NOX_Square', 'NOX_RM', 'NOX_AGE', 'NOX_DIS', 'NOX_RAD', 'NOX_TAX', 'NOX_PTRATIO', 'NOX_B', 'NOX_LSTAT', 'RM_Square', 'RM_AGE', 'RM_DIS', 'RM_RAD', 'RM_TAX', 'RM_PTRATIO', 'RM_B', 'RM_LSTAT', 'AGE_Square', 'AGE_DIS', 'AGE_RAD', 'AGE_TAX', 'AGE_PTRATIO', 'AGE_B', 'AGE_LSTAT', 'DIS_Square', 'DIS_RAD', 'DIS_TAX', 'DIS_PTR

In [ ]:
Xpoly.shape

(506, 105)

In [ ]:
regr2 = linear_model.LinearRegression()

regr2

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
regr2.fit(Xpoly,Y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
regr2.score(Xpoly,Y)

0.9289668414388449

In [ ]:
regr2.adjR2, regr2.R2 = AdjustedRSquare(regr2,Xpoly,Y)
print (regr2.adjR2, regr2.R2)


0.9103206373165417 0.9289668414388449


In [ ]:
np.round(ReturnPValue(regr2,Xpoly,Y),1)

/Users/pradmishra/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]


array([0.5, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0.1, 0.1, 0. ,
       0. , 0. , 0.1, 0. , 0. , 0.2, 0. , 0.2, 0.2, 0.3, 0.2, 0. , 0. ,
       0. , 0.1, 0.1, 0.1, 0. , 0.2, 0.5, 0.1, 0.2, 0. , 0.1, 0.2, 0.1,
       0. , 0.5, 0.2, 0. , 0.1, 0. , 0.3, 0.2, 0.3, 0.1, 0.1, 0. , 0. ,
       0. , 0.4, 0.2, 0.4, 0.5, 0. , 0.1, 0.1, 0.2, 0.4, 0. , 0. , 0.1,
       0. , 0. , 0.2, 0. , 0. , 0. , 0.4, 0.3, 0. , 0. , 0.1, 0. , 0.3,
       0.3, 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. , 0. ,
       0. , 0.1, 0.4, 0. , 0. , 0. , 0.3, 0.2, 0.2, 0.1, 0.4, 0. , 0.1,
       0. ])

In [ ]:
print (polyfeaturenames)

['intercept', 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'CRIM_Square', 'CRIM_ZN', 'CRIM_INDUS', 'CRIM_CHAS', 'CRIM_NOX', 'CRIM_RM', 'CRIM_AGE', 'CRIM_DIS', 'CRIM_RAD', 'CRIM_TAX', 'CRIM_PTRATIO', 'CRIM_B', 'CRIM_LSTAT', 'ZN_Square', 'ZN_INDUS', 'ZN_CHAS', 'ZN_NOX', 'ZN_RM', 'ZN_AGE', 'ZN_DIS', 'ZN_RAD', 'ZN_TAX', 'ZN_PTRATIO', 'ZN_B', 'ZN_LSTAT', 'INDUS_Square', 'INDUS_CHAS', 'INDUS_NOX', 'INDUS_RM', 'INDUS_AGE', 'INDUS_DIS', 'INDUS_RAD', 'INDUS_TAX', 'INDUS_PTRATIO', 'INDUS_B', 'INDUS_LSTAT', 'CHAS_Square', 'CHAS_NOX', 'CHAS_RM', 'CHAS_AGE', 'CHAS_DIS', 'CHAS_RAD', 'CHAS_TAX', 'CHAS_PTRATIO', 'CHAS_B', 'CHAS_LSTAT', 'NOX_Square', 'NOX_RM', 'NOX_AGE', 'NOX_DIS', 'NOX_RAD', 'NOX_TAX', 'NOX_PTRATIO', 'NOX_B', 'NOX_LSTAT', 'RM_Square', 'RM_AGE', 'RM_DIS', 'RM_RAD', 'RM_TAX', 'RM_PTRATIO', 'RM_B', 'RM_LSTAT', 'AGE_Square', 'AGE_DIS', 'AGE_RAD', 'AGE_TAX', 'AGE_PTRATIO', 'AGE_B', 'AGE_LSTAT', 'DIS_Square', 'DIS_RAD', 'DIS_TAX', 'DIS_PTR

In [ ]:
regr2.MAPE, regr2.MSSE = ErrorMetric(regr2,Xpoly,Y)
print (regr2.MAPE, regr2.MSSE)

9.39697020374462 5.996587718103243


In [ ]:
print (regr.coef_)
print (regr2.coef_)

[-6.29027110e-02  2.18922198e-02  9.31070144e-02  2.93069696e+00
 -1.11960674e+01  3.09254199e+00  4.37965130e-02 -1.11353972e-02
 -1.19246843e+00  9.29090571e-03 -6.24382465e-01  3.28799043e+00
  1.77845653e+00  1.52138497e+00  1.03079931e+00  1.85912890e+00
  1.95965565e+00  8.05104573e+00]
[-3.19540199e-12  1.59720526e+01  1.49563747e+01  9.23418581e+00
  3.80461129e+12 -1.88273323e+00  3.51595276e+00 -2.08801570e+00
 -2.45756745e+00  5.06695557e+00 -9.98182678e+00  1.16366577e+00
  1.40481567e+00 -3.14845276e+00  1.09619141e-01  4.34020386e+01
  2.31938171e+01  5.42572021e+00 -7.23785400e-01  8.54125977e-01
 -7.93746948e-01 -1.75195312e+00  1.58265991e+01 -3.45992889e+01
  7.23960114e+00 -2.59765625e-01  1.29612732e+00 -2.75146484e-01
 -8.61022949e-01 -3.55010986e-01 -2.86883545e+00  2.30834961e-01
  3.55224609e-02 -5.19226074e-01 -1.04397583e+00  2.51507568e+00
 -3.43261719e-01  1.40032959e+00 -7.75146484e-01  1.88409424e+00
  1.65405273e-02  9.12719727e-01  1.40759277e+00  7.0052

In [ ]:
# Null: beta estimates are = 0
# Alt : beta estimates are != 0

In [ ]:
# to restrict the model overfitting
# regularized regression models

# LASSO - L1
# Ridge - L2

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
XTrain, XTest, YTrain, YTest = cv.train_test_split(Xpoly,Y, test_size=0.25,random_state=1234)

In [ ]:
XTrain.shape, XTest.shape, YTrain.shape, YTest.shape

((379, 105), (127, 105), (379,), (127,))

In [ ]:
lassoMod = Lasso(alpha=0.9)

In [ ]:
lassoMod.fit(XTrain,YTrain)

Lasso(alpha=0.9, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
print (lassoMod.coef_)

[ 0.         -0.          0.         -0.          0.         -0.
  1.60399752 -0.         -0.         -0.         -0.         -0.70436791
  0.         -3.95772244 -0.07577914  0.         -0.          0.
 -0.         -0.         -0.          0.         -0.         -0.
  0.          0.         -0.          0.04931477 -0.          0.
  0.          0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.          0.         -0.
 -0.         -0.          0.         -0.         -0.          0.
  0.         -0.          0.19527659 -0.         -0.          0.
 -0.          0.          0.          0.          0.         -0.
 -0.         -0.         -0.          0.         -0.         -0.
 -0.          0.         -0.          0.74651837 -0.          0.
 -0.         -0.03396626 -1.01960633  0.         -0.21098532  0.
  0.         -0.         -0.         -0.         -0.         -0.
 -0.          0.          0.          0.         -0.          0.
 -0.         -0. 

In [ ]:
lassoMod.coef_[((lassoMod.coef_)!=0)]

array([ 1.60399752, -0.70436791, -3.95772244, -0.07577914,  0.04931477,
        0.19527659,  0.74651837, -0.03396626, -1.01960633, -0.21098532,
       -0.25900908])

In [ ]:
lassoMod = Lasso(alpha=1)
lassoMod.fit(XTrain,YTrain)
print (lassoMod.coef_)

[ 0.         -0.          0.         -0.          0.         -0.
  1.56690689 -0.         -0.         -0.         -0.         -0.65876025
  0.         -3.84571096 -0.07516     0.         -0.          0.
 -0.         -0.         -0.          0.         -0.         -0.
  0.          0.         -0.          0.06359877 -0.          0.
  0.          0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.          0.         -0.
 -0.         -0.          0.         -0.         -0.          0.
  0.         -0.          0.19172019 -0.         -0.          0.
 -0.          0.          0.          0.          0.         -0.
 -0.         -0.         -0.          0.         -0.         -0.
 -0.          0.         -0.          0.77647613 -0.          0.
 -0.         -0.         -0.99777999  0.         -0.12426401  0.
  0.         -0.         -0.         -0.          0.         -0.
 -0.          0.          0.          0.         -0.          0.
 -0.         -0. 

In [ ]:
lassoMod.coef_[((lassoMod.coef_)!=0)]

array([ 1.56690689, -0.65876025, -3.84571096, -0.07516   ,  0.06359877,
        0.19172019,  0.77647613, -0.99777999, -0.12426401, -0.26425064])

In [ ]:
names = [polyfeaturenames[i] for i in list(np.where(lassoMod.coef_ != 0.0)[0])]
names

['RM',
 'PTRATIO',
 'LSTAT',
 'CRIM_Square',
 'ZN_Square',
 'CHAS_Square',
 'RM_Square',
 'RM_PTRATIO',
 'RM_LSTAT',
 'B_Square']

In [ ]:
lassoMod.MAPE, lassoMod.MSSE = ErrorMetric(lassoMod, XTrain, YTrain)

In [ ]:
print (lassoMod.MAPE, lassoMod.MSSE)

15.060789081414336 19.778887972486704


In [ ]:
lassoMod.MAPE, lassoMod.MSSE = ErrorMetric(lassoMod, XTest, YTest)

In [ ]:
print (lassoMod.MAPE, lassoMod.MSSE)

18.83289459332976 21.269395820537735


In [ ]:
from sklearn.linear_model import Ridge


In [ ]:
ridgeMod1 = Ridge(alpha=1)
ridgeMod1.fit(XTrain,YTrain)
names = [polyfeaturenames[i] for i in list(np.where(ridgeMod1.coef_ != 0.0)[0])]
print(names)
ridgeMod1.MAPE, ridgeMod1.MSSE = ErrorMetric(ridgeMod1, XTrain, YTrain)
print (ridgeMod1.MAPE, ridgeMod1.MSSE)


['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'CRIM_Square', 'CRIM_ZN', 'CRIM_INDUS', 'CRIM_CHAS', 'CRIM_NOX', 'CRIM_RM', 'CRIM_AGE', 'CRIM_DIS', 'CRIM_RAD', 'CRIM_TAX', 'CRIM_PTRATIO', 'CRIM_B', 'CRIM_LSTAT', 'ZN_Square', 'ZN_INDUS', 'ZN_CHAS', 'ZN_NOX', 'ZN_RM', 'ZN_AGE', 'ZN_DIS', 'ZN_RAD', 'ZN_TAX', 'ZN_PTRATIO', 'ZN_B', 'ZN_LSTAT', 'INDUS_Square', 'INDUS_CHAS', 'INDUS_NOX', 'INDUS_RM', 'INDUS_AGE', 'INDUS_DIS', 'INDUS_RAD', 'INDUS_TAX', 'INDUS_PTRATIO', 'INDUS_B', 'INDUS_LSTAT', 'CHAS_Square', 'CHAS_NOX', 'CHAS_RM', 'CHAS_AGE', 'CHAS_DIS', 'CHAS_RAD', 'CHAS_TAX', 'CHAS_PTRATIO', 'CHAS_B', 'CHAS_LSTAT', 'NOX_Square', 'NOX_RM', 'NOX_AGE', 'NOX_DIS', 'NOX_RAD', 'NOX_TAX', 'NOX_PTRATIO', 'NOX_B', 'NOX_LSTAT', 'RM_Square', 'RM_AGE', 'RM_DIS', 'RM_RAD', 'RM_TAX', 'RM_PTRATIO', 'RM_B', 'RM_LSTAT', 'AGE_Square', 'AGE_DIS', 'AGE_RAD', 'AGE_TAX', 'AGE_PTRATIO', 'AGE_B', 'AGE_LSTAT', 'DIS_Square', 'DIS_RAD', 'DIS_TAX', 'DIS_PTRATIO', 'DIS_B

In [ ]:
ridgeMod1 = Ridge(alpha=0)
ridgeMod1.fit(XTrain,YTrain)
names = [polyfeaturenames[i] for i in list(np.where(ridgeMod1.coef_ != 0.0)[0])]
print(names)
ridgeMod1.MAPE, ridgeMod1.MSSE = ErrorMetric(ridgeMod1, XTrain, YTrain)
print (ridgeMod1.MAPE, ridgeMod1.MSSE)


['intercept', 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'CRIM_Square', 'CRIM_ZN', 'CRIM_INDUS', 'CRIM_CHAS', 'CRIM_NOX', 'CRIM_RM', 'CRIM_AGE', 'CRIM_DIS', 'CRIM_RAD', 'CRIM_TAX', 'CRIM_PTRATIO', 'CRIM_B', 'CRIM_LSTAT', 'ZN_Square', 'ZN_INDUS', 'ZN_CHAS', 'ZN_NOX', 'ZN_RM', 'ZN_AGE', 'ZN_DIS', 'ZN_RAD', 'ZN_TAX', 'ZN_PTRATIO', 'ZN_B', 'ZN_LSTAT', 'INDUS_Square', 'INDUS_CHAS', 'INDUS_NOX', 'INDUS_RM', 'INDUS_AGE', 'INDUS_DIS', 'INDUS_RAD', 'INDUS_TAX', 'INDUS_PTRATIO', 'INDUS_B', 'INDUS_LSTAT', 'CHAS_Square', 'CHAS_NOX', 'CHAS_RM', 'CHAS_AGE', 'CHAS_DIS', 'CHAS_RAD', 'CHAS_TAX', 'CHAS_PTRATIO', 'CHAS_B', 'CHAS_LSTAT', 'NOX_Square', 'NOX_RM', 'NOX_AGE', 'NOX_DIS', 'NOX_RAD', 'NOX_TAX', 'NOX_PTRATIO', 'NOX_B', 'NOX_LSTAT', 'RM_Square', 'RM_AGE', 'RM_DIS', 'RM_RAD', 'RM_TAX', 'RM_PTRATIO', 'RM_B', 'RM_LSTAT', 'AGE_Square', 'AGE_DIS', 'AGE_RAD', 'AGE_TAX', 'AGE_PTRATIO', 'AGE_B', 'AGE_LSTAT', 'DIS_Square', 'DIS_RAD', 'DIS_TAX', 'DIS_PTR

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
XTrain, XTest, YTrain, YTest = cv.train_test_split(Xpoly,Y, test_size=0.25,random_state=1234)

In [ ]:
XTrain.shape, XTest.shape, YTrain.shape, YTest.shape

((379, 105), (127, 105), (379,), (127,))

In [ ]:
from sklearn.model_selection import ShuffleSplit

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
def performance_metric(y_true, y_predict):
    score = r2_score(y_true, y_predict)
    return score

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
def fit_model(X,y):
    cv_sets = ShuffleSplit(XTrain.shape[0],
                           test_size=0.20,
                           random_state=1234)
    ridgeModel = Ridge()
    params = {'alpha':list(range(0,12)),
             'solver' : ('auto', 
                         'svd', 
                         'cholesky', 
                         'lsqr', 
                         'sparse_cg', 
                         'sag', 
                         'saga')}
    scoring_func = make_scorer(performance_metric)
    grid = GridSearchCV(ridgeModel,params,scoring_func,cv=cv_sets)
    grid = grid.fit(X,y)
    return grid.best_estimator_

In [ ]:
final_model = fit_model(XTrain,YTrain)

In [ ]:
ErrorMetric(final_model, XTrain, YTrain)

(9.377393843857805, 6.563589918182345)

In [ ]:
ErrorMetric(final_model, XTest, YTest)

(12.625047369373233, 10.922663960766162)

In [ ]:
regr2.adjR2, regr2.R2 = AdjustedRSquare(final_model, XTrain, YTrain)
print (regr2.adjR2, regr2.R2)

0.8839842461082027 0.9162108444114797


In [ ]:
regr2.adjR2, regr2.R2 = AdjustedRSquare(final_model, XTest, YTest)
print (regr2.adjR2, regr2.R2)

0.3591500323173351 0.8931916720528892
